# US Census Block Group FIPS codes

Download .shp file from https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_bg_500k.zip (via https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.2020.html)

In [59]:
!wget https://www2.census.gov/geo/tiger/GENZ2021/shp/cb_2021_us_bg_500k.zip

--2023-07-24 18:08:07--  https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_bg_500k.zip
Resolving www2.census.gov (www2.census.gov)... 2600:1400:d:492::208c, 2600:1400:d:4a5::208c, 23.4.233.168
Connecting to www2.census.gov (www2.census.gov)|2600:1400:d:492::208c|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘cb_2020_us_bg_500k.zip’

cb_2020_us_bg_500k.     [        <=>         ]  99.38M  6.49MB/s    in 15s     

2023-07-24 18:08:22 (6.54 MB/s) - ‘cb_2020_us_bg_500k.zip’ saved [104212096]



In [60]:
!unzip cb_2020_us_bg_500k.zip

Archive:  cb_2020_us_bg_500k.zip
  inflating: cb_2020_us_bg_500k.shp.ea.iso.xml  
  inflating: cb_2020_us_bg_500k.shp.iso.xml  
  inflating: cb_2020_us_bg_500k.shp  
  inflating: cb_2020_us_bg_500k.shx  
  inflating: cb_2020_us_bg_500k.dbf  
  inflating: cb_2020_us_bg_500k.prj  
 extracting: cb_2020_us_bg_500k.cpg  


In [61]:
!pip install shpinfo

In [114]:
!shpinfo /Users/me/Downloads/cb_2021_us_bg_500k.shx


Fields
------

Field Name                         |Field Type    |  Field Length|Decimal Length
--------------------------------------------------------------------------------
STATEFP                            |Character     |             2|             0
COUNTYFP                           |Character     |             3|             0
TRACTCE                            |Character     |             6|             0
BLKGRPCE                           |Character     |             1|             0
AFFGEOID                           |Character     |            21|             0
GEOID                              |Character     |            12|             0
NAME                               |Character     |           100|             0
NAMELSAD                           |Character     |           100|             0
LSAD                               |Character     |             2|             0
ALAND                              |Number        |            14|             0
AWATER      

# Load the shx file with duckdb spatial extension

https://duckdb.org/2023/04/28/spatial.html

In [63]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## 

In [64]:
%%sql
INSTALL spatial;

,Success


In [65]:
%%sql
LOAD spatial;

,Success


In [116]:
%%sql
-- Create a table to hold the census block groups
CREATE TABLE census_ AS SELECT statefp, countyfp, tractce, blkgrpce, affgeoid, geoid, name, namelsad, lsad, aland, awater, ST_GeomFromWKB(wkb_geometry) AS geom 
FROM ST_Read('/Users/me/Downloads/cb_2021_us_bg_500k.shx');

,Count
0,242384


In [117]:
%%sql
DESCRIBE census_;

,column_name,column_type,null,key,default,extra
0,STATEFP,VARCHAR,YES,NaN,NaN,NaN
1,COUNTYFP,VARCHAR,YES,NaN,NaN,NaN
2,TRACTCE,VARCHAR,YES,NaN,NaN,NaN
3,BLKGRPCE,VARCHAR,YES,NaN,NaN,NaN
4,AFFGEOID,VARCHAR,YES,NaN,NaN,NaN
5,GEOID,VARCHAR,YES,NaN,NaN,NaN
6,NAME,VARCHAR,YES,NaN,NaN,NaN
7,NAMELSAD,VARCHAR,YES,NaN,NaN,NaN
8,LSAD,VARCHAR,YES,NaN,NaN,NaN
9,ALAND,BIGINT,YES,NaN,NaN,NaN


In [118]:
%%sql 
SELECT * FROM census_ LIMIT 10;

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,GEOID,NAME,NAMELSAD,LSAD,ALAND,AWATER,geom
0,01,053,970700,1,1500000US010539707001,010539707001,1,Block Group 1,BG,34838783,554870,"[0, 2, 114, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
1,04,013,216844,2,1500000US040132168442,040132168442,2,Block Group 2,BG,2729117,0,"[0, 2, 240, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
2,06,073,007303,2,1500000US060730073032,060730073032,2,Block Group 2,BG,155982,0,"[0, 2, 112, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
3,06,037,297601,3,1500000US060372976013,060372976013,3,Block Group 3,BG,442073,0,"[0, 2, 240, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
4,06,001,428302,3,1500000US060014283023,060014283023,3,Block Group 3,BG,647392,130654,"[0, 2, 176, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
5,06,041,130204,1,1500000US060411302041,060411302041,1,Block Group 1,BG,877599,3843644,"[0, 2, 161, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
6,06,059,062702,5,1500000US060590627025,060590627025,5,Block Group 5,BG,372747,14119,"[0, 2, 192, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
7,09,011,702900,2,1500000US090117029002,090117029002,2,Block Group 2,BG,2695713,2037940,"[0, 2, 161, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
8,11,001,009700,1,1500000US110010097001,110010097001,1,Block Group 1,BG,89205,0,"[0, 2, 112, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."
9,11,001,001004,1,1500000US110010010041,110010010041,1,Block Group 1,BG,277610,0,"[0, 2, 112, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0..."


In [119]:
%%sql
SELECT GEOID, ST_X(ST_Centroid(geom)) AS longitude, ST_Y(ST_Centroid(geom)) AS latitude FROM census_ LIMIT 10;

,GEOID,longitude,latitude
0,010539707001,-87.420878,-87.420878
1,040132168442,-111.915209,-111.915209
2,060730073032,-117.251575,-117.251575
3,060372976013,-118.286036,-118.286036
4,060014283023,-122.244139,-122.244139
5,060411302041,-122.499548,-122.499548
6,060590627025,-117.874299,-117.874299
7,090117029002,-71.995052,-71.995052
8,110010097001,-76.991666,-76.991666
9,110010010041,-77.086785,-77.086785


In [120]:
%%sql
SELECT GEOID, ST_X(ST_Centroid(geom)) AS longitude FROM census_ LIMIT 10;

,GEOID,longitude
0,010539707001,-87.420878
1,040132168442,-111.915209
2,060730073032,-117.251575
3,060372976013,-118.286036
4,060014283023,-122.244139
5,060411302041,-122.499548
6,060590627025,-117.874299
7,090117029002,-71.995052
8,110010097001,-76.991666
9,110010010041,-77.086785


In [121]:
%%sql
SELECT GEOID, ST_Y(ST_Centroid(geom)) AS latitude FROM census_ LIMIT 10;

,GEOID,latitude
0,010539707001,31.011434
1,040132168442,33.663256
2,060730073032,32.739362
3,060372976013,33.717620
4,060014283023,37.746107
5,060411302041,37.870586
6,060590627025,33.596254
7,090117029002,41.329726
8,110010097001,38.830456
9,110010010041,38.957000


In [122]:
%%sql
WITH census_block_centroids AS (
  SELECT ST_transform(ST_Centroid(geom), 'EPSG:4269', 'EPSG:4326') AS centroid FROM census_
),
centroids_lat_long AS (
  SELECT 
  ST_X(centroid) AS longitude, 
  ST_Y(centroid) AS latitude FROM census_block_centroids
)
SELECT * FROM centroids_lat_long LIMIT 10;

,longitude,latitude
0,-87.420878,-87.420878
1,-111.915209,-111.915209
2,-117.251575,-117.251575
3,-118.286036,-118.286036
4,-122.244139,-122.244139
5,-122.499548,-122.499548
6,-117.874299,-117.874299
7,-71.995052,-71.995052
8,-76.991666,-76.991666
9,-77.086785,-77.086785


In [124]:
%%sql
CREATE TABLE census_all_ AS SELECT statefp, countyfp, tractce, blkgrpce, affgeoid, geoid, name, namelsad, lsad, aland, awater, ST_GeomFromWKB(wkb_geometry) AS geom 
FROM ST_Read('/Users/me/Downloads/cb_2021_us_bg_500k.shx');

,Count
0,242384


In [126]:
%%sql 
CREATE TABLE census_all_long_ AS 
SELECT GEOID, ST_X(ST_Centroid(geom)) AS longitude, statefp, countyfp, tractce, blkgrpce, affgeoid, name, namelsad, lsad, aland
FROM census_all_;

,Count
0,242384


In [127]:
%%sql 
CREATE TABLE census_only_lat_ AS 
SELECT GEOID, ST_Y(ST_Centroid(geom)) AS latitude 
FROM census_;

,Count
0,242384


In [128]:
%%sql 
CREATE TABLE census_all_latlong_ AS
SELECT 
  census_all_long_.*,
  census_only_lat_.latitude,
FROM census_all_long_
JOIN census_only_lat_ ON census_only_lat_.GEOID = census_all_long_.GEOID;

,Count
0,242384


In [129]:
%%sql
COPY (
  SELECT * FROM census_all_latlong_
) TO '/Users/me/Downloads/census_block_groups_latitude_longitude.parquet' (COMPRESSION ZSTD);

,Count
0,242384


In [130]:
import polars as pl

In [131]:
pl.read_parquet('/Users/me/Downloads/census_block_groups_latitude_longitude.parquet')

GEOID,longitude,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,AFFGEOID,NAME,NAMELSAD,LSAD,ALAND,latitude
str,f64,str,str,str,str,str,str,str,str,i64,f64
"""320030016112""",-115.069852,"""32""","""003""","""001611""","""2""","""1500000US32003…","""2""","""Block Group 2""","""BG""",663342,36.148129
"""320050011001""",-119.732187,"""32""","""005""","""001100""","""1""","""1500000US32005…","""1""","""Block Group 1""","""BG""",2022004,38.883508
"""320030019011""",-115.128058,"""32""","""003""","""001901""","""1""","""1500000US32003…","""1""","""Block Group 1""","""BG""",209570,36.141856
"""320310035102""",-119.737654,"""32""","""031""","""003510""","""2""","""1500000US32031…","""2""","""Block Group 2""","""BG""",594717,39.588675
"""320030002042""",-115.1666,"""32""","""003""","""000204""","""2""","""1500000US32003…","""2""","""Block Group 2""","""BG""",1144494,36.150667
"""320310035173""",-119.67395,"""32""","""031""","""003517""","""3""","""1500000US32031…","""3""","""Block Group 3""","""BG""",625323,39.61295
"""320310032073""",-119.750902,"""32""","""031""","""003207""","""3""","""1500000US32031…","""3""","""Block Group 3""","""BG""",25566087,39.305116
"""350619703062""",-106.668706,"""35""","""061""","""970306""","""2""","""1500000US35061…","""2""","""Block Group 2""","""BG""",8828015,34.778454
"""350280005004""",-106.290577,"""35""","""028""","""000500""","""4""","""1500000US35028…","""4""","""Block Group 4""","""BG""",53076356,35.80287
